In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import os


In [3]:
# Define paths
data_dir = './dataset'

# Define transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to match model input size
    transforms.ToTensor()         # Convert images to tensors
])

# Load dataset using ImageFolder
dataset = datasets.ImageFolder(root=data_dir, transform=transform)


train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Split dataset
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# Define DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Testing samples: {len(test_dataset)}")



Training samples: 496
Validation samples: 165
Testing samples: 167


In [5]:
# Load a pretrained ResNet model
model = models.resnet18(pretrained=True)

# Modify the final layer for classification
num_classes = len(dataset.classes)  # Number of classes based on folder names
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


d:\Mazay-Wali-Folder\Machine Learning\Crops-Classification\env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Mazay-Wali-Folder\Machine Learning\Crops-Classification\env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [7]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {running_loss/len(train_loader):.4f}, "
          f"Val Loss: {val_loss/len(val_loader):.4f}, "
          f"Val Accuracy: {correct/total:.4f}")


Epoch 1/10, Train Loss: 2.3471, Val Loss: 3.2861, Val Accuracy: 0.3193
Epoch 2/10, Train Loss: 1.1876, Val Loss: 2.0301, Val Accuracy: 0.4036
Epoch 3/10, Train Loss: 0.7741, Val Loss: 2.2004, Val Accuracy: 0.4699
Epoch 4/10, Train Loss: 0.4822, Val Loss: 2.3407, Val Accuracy: 0.4819
Epoch 5/10, Train Loss: 0.2879, Val Loss: 1.2542, Val Accuracy: 0.6265
Epoch 6/10, Train Loss: 0.2250, Val Loss: 1.2280, Val Accuracy: 0.6386
Epoch 7/10, Train Loss: 0.1594, Val Loss: 1.5598, Val Accuracy: 0.5904
Epoch 8/10, Train Loss: 0.1268, Val Loss: 1.5994, Val Accuracy: 0.5904
Epoch 9/10, Train Loss: 0.0819, Val Loss: 0.8998, Val Accuracy: 0.7410
Epoch 10/10, Train Loss: 0.0584, Val Loss: 1.1825, Val Accuracy: 0.6566


In [8]:
torch.save(model.state_dict(), 'res_crop_classifier_cnn.pth')

In [9]:
def calculate_accuracy(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculations for evaluation
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Get predictions
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)  # Get the index of the max probability
            
            # Update counts
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total * 100  # Convert to percentage
    return accuracy


In [10]:
train_accuracy = calculate_accuracy(model, train_loader, device)
val_accuracy = calculate_accuracy(model, val_loader, device)

print(f"Training Accuracy: {train_accuracy:.2f}%")
print(f"Validation Accuracy: {val_accuracy:.2f}%")

Training Accuracy: 97.58%
Validation Accuracy: 65.66%
